In [ ]:
#медленная версия
import os
from tqdm import tqdm
from rapidfuzz.distance import DamerauLevenshtein


def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())


def load_queries(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read().splitlines()


def find_extra(word, dict_word):
    for i in range(len(word)):
        new_word = word[:i] + word[i + 1:]
        if DamerauLevenshtein.distance(new_word, dict_word, score_cutoff=3) == 1:
            return new_word


def find_missed(word, dict_word):
    for i in range(len(dict_word)):
        new_word = word[:i] + dict_word[i] + word[i:]
        if DamerauLevenshtein.distance(new_word, dict_word, score_cutoff=3) == 1:
            return new_word


def find_operation(word, dict_word):
    for i in range(len(word)):
        new_word = word[:i] + word[i + 1:]
        if DamerauLevenshtein.distance(new_word, dict_word, score_cutoff=3) == 1:
            return new_word
    #перестановка
    for i in range(len(word) - 1):
        new_word = list(word)
        new_word[i], new_word[i + 1] = new_word[i + 1], new_word[i]
        new_word = ''.join(new_word)
        if DamerauLevenshtein.distance(new_word, dict_word, score_cutoff=3) == 1:
            return new_word
    #замена
    for i in range(len(word)):
        new_word = word[:i] + dict_word[i] + word[i + 1:]
        if DamerauLevenshtein.distance(new_word, dict_word, score_cutoff=3) == 1:
            return new_word
    
            

def process_queries(dictionary, queries, output_file):
    with open(output_file, 'w', encoding='utf-8') as out_f:
        for word in tqdm(queries, desc='Обработка запросов', unit='запрос'):
            if word in dictionary:
                out_f.write(f"{word} 0\n")
                continue

            candidate = ''
            need_write = True
            for dict_word in dictionary:
                distance = DamerauLevenshtein.distance(word, dict_word, score_cutoff=3)
                if distance == 1:
                    out_f.write(f"{word} 1 {dict_word}\n")
                    need_write = False
                    break
                if distance == 2:
                    candidate = dict_word
                    
            if candidate and need_write:
                need_write = False
                dict_word = candidate
                if len(word) > len(dict_word):
                    correction = find_extra(word, dict_word)
                elif len(word) < len(dict_word):
                    correction = find_missed(word, dict_word)
                    if correction is None:
                        correction = find_operation(word, dict_word)
                else:
                    correction = find_operation(word, dict_word)
                out_f.write(f"{word} 2 {correction} {dict_word}\n")
                
            if need_write:
                out_f.write(f"{word} 3+\n")

# Пути к файлам
dictionary_file = 'data/548_сorrect_all_typos/dict.txt'
queries_file = 'data/548_сorrect_all_typos/queries.txt'
output_file = 'data/548_сorrect_all_typos/answer.txt'

dictionary = load_dictionary(dictionary_file)
queries = load_queries(queries_file)
# Выполнение
process_queries(dictionary, queries, output_file)